In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [2]:
df= pd.read_csv('고민분류용df2.csv', encoding='utf-8')

,Unnamed: 0,text,label
0,0,공황장애 극복 방법 치료 가능할까 요 공황장애 극복 방법 치료 도움 되는것 줌 알려...,9
1,1,공황장애 증상 일까 요 제 작년 여름 에 숨쉬기 가 힘들고 가슴 이 답답하고 땀 이...,9
2,2,공황장애 일까 요 안녕하세요 제 가 요즘 일 을 해야되 서 일 을 하게되 면 심장 ...,9
3,3,공황장애 강박증 안녕하세요 살 여자 입니다 작 년월 에 갑작스러운 공황발작 이후 로...,9
4,4,공황장애 요 ㅜㅜ 좋은 꿈 을 정도 꾼다면 나쁜 꿈 은 정도 되는데 무슨 의미 가 ...,9
...,...,...,...
17315,17315,여드름 피부 고민 좀 들어주세요 안녕하세요 살 여자 입니다 저 는 여드름 을 달 고...,4
17316,17316,피부 에 고민 이 많은 살 여자 피부 에 고민 이 많은 살 여자 에요 초등학교 학년...,4
17317,17317,피부 고민 탄력성 안녕하세요 피부 트러블 이 고민 인 한 여학생 입니다 제 가 이마...,4
17318,17318,안녕하세요 여드름 피부 때문 에 고민 인 학생 입니다 천연 비누 추천 좀 해주세요,4


In [6]:
import pickle

with open('index_to_word_byOkt.pickle', 'rb') as f :
    index_to_word = pickle.load(f)
with open('index_to_noums_byOkt.pickle', 'rb') as f :
    index_to_noums = pickle.load(f)
with open('word_to_index_byOkt.pickle', 'rb') as f :
    word_to_index = pickle.load(f)
with open('noums_to_index_byOkt.pickle', 'rb') as f :
    noums_to_index = pickle.load(f)

In [10]:
df = df.dropna()

In [13]:
df.text = df.text.apply(lambda x: x.split())

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [27]:
#단어 -> 인덱스 변환
def n_to_int(x):
    pad_index = []
    for i in x:
        if i in noums_to_index.keys():
            pad_index.append(noums_to_index[i])
    return pad_index

#morph -> 인덱스 변환
def w_to_int(x):
    pad_index = []
    for i in x:
        if i in word_to_index.keys():
            pad_index.append(word_to_index[i])
    return pad_index

In [28]:
nom_X = df.text.apply(lambda x: n_to_int(x))
mor_X = df.text.apply(lambda x: w_to_int(x))

In [43]:
from keras.preprocessing.sequence import pad_sequences

X_nom =pad_sequences(nom_X, maxlen=70, padding='post', truncating='post')
X_mor =pad_sequences(mor_X, maxlen=70, padding='post', truncating='post')


Using TensorFlow backend.


In [45]:
Y = df.label.values

In [46]:
from sklearn.model_selection import train_test_split

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_nom, Y, test_size = 0.1, random_state = 42)
x_m_train, x_m_test, y_m_train, y_m_test = train_test_split(X_mor, Y, test_size = 0.1, random_state = 42)

In [49]:
from keras.utils import to_categorical

y_n_train = to_categorical(y_n_train)
y_n_test = to_categorical(y_n_test)

y_m_train = to_categorical(y_m_train)
y_m_test = to_categorical(y_m_test)


In [50]:
x_n_train.shape, y_n_train.shape , x_m_train.shape, y_m_train.shape

((15580, 70), (15580, 10), (15580, 70), (15580, 10))

In [57]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.layers import LSTM
model1 = Sequential()
model1.add(Embedding(9999, 32,input_length=70))
model1.add(LSTM(64, return_sequences= True))
model1.add(LSTM(32))
model1.add(Dense(10, activation="softmax"))

In [58]:
model1.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [59]:
history = model1.fit(x_n_train, y_n_train, batch_size =128, epochs =8 , validation_split =0.2)

Train on 12464 samples, validate on 3116 samples
Epoch 1/8
12464/12464 [==============================] - 11s 854us/step - loss: 2.0316 - acc: 0.2236 - val_loss: 1.6546 - val_acc: 0.3206
Epoch 2/8
12464/12464 [==============================] - 9s 744us/step - loss: 1.1850 - acc: 0.5195 - val_loss: 0.9679 - val_acc: 0.5616
Epoch 3/8
12464/12464 [==============================] - 9s 758us/step - loss: 0.7975 - acc: 0.6823 - val_loss: 0.7340 - val_acc: 0.7535
Epoch 4/8
12464/12464 [==============================] - 9s 742us/step - loss: 0.6089 - acc: 0.7833 - val_loss: 0.6387 - val_acc: 0.7542
Epoch 5/8
12464/12464 [==============================] - 9s 746us/step - loss: 0.5127 - acc: 0.8305 - val_loss: 0.5422 - val_acc: 0.8415
Epoch 6/8
12464/12464 [==============================] - 10s 777us/step - loss: 0.4092 - acc: 0.8821 - val_loss: 0.4819 - val_acc: 0.8633
Epoch 7/8
12464/12464 [==============================] - 10s 765us/step - loss: 0.3338 - acc: 0.9015 - val_loss: 0.4198 - val_a

In [64]:
history = model1.fit(x_n_train, y_n_train, batch_size =128, epochs =1 , validation_split =0.2)

Train on 12464 samples, validate on 3116 samples
Epoch 1/1
12464/12464 [==============================] - 9s 744us/step - loss: 0.2772 - acc: 0.9138 - val_loss: 0.4175 - val_acc: 0.8777


In [62]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.layers import LSTM
model2 = Sequential()
model2.add(Embedding(9997, 32,input_length=70))
model2.add(SimpleRNN(64, return_sequences= True))
model2.add(SimpleRNN(32))
model2.add(Dense(10, activation="softmax"))

In [63]:
model2.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model2.fit(x_m_train, y_m_train, batch_size =128, epochs =10 , validation_split =0.2)

Train on 12464 samples, validate on 3116 samples
Epoch 1/10
12464/12464 [==============================] - 11s 857us/step - loss: 2.0134 - acc: 0.2383 - val_loss: 1.6438 - val_acc: 0.3052
Epoch 2/10
12464/12464 [==============================] - 9s 746us/step - loss: 1.5248 - acc: 0.3544 - val_loss: 1.5726 - val_acc: 0.3405
Epoch 3/10
12464/12464 [==============================] - 9s 743us/step - loss: 1.4068 - acc: 0.3962 - val_loss: 1.3029 - val_acc: 0.4220
Epoch 4/10
12464/12464 [==============================] - 9s 723us/step - loss: 1.4623 - acc: 0.4085 - val_loss: 1.4220 - val_acc: 0.4146
Epoch 5/10
12464/12464 [==============================] - 9s 725us/step - loss: 1.3099 - acc: 0.4292 - val_loss: 1.3091 - val_acc: 0.4178
Epoch 6/10
12464/12464 [==============================] - 9s 742us/step - loss: 1.2962 - acc: 0.4190 - val_loss: 1.3134 - val_acc: 0.4117
Epoch 7/10
12464/12464 [==============================] - 9s 737us/step - loss: 1.3026 - acc: 0.4450 - val_loss: 1.3578 - 

In [65]:
model1.evaluate(x_n_test, y_n_test)

1732/1732 [==============================] - 1s 305us/step


[0.4155722647285902, 0.874133949191686]

In [66]:
model2.evaluate(x_m_test, y_m_test)

1732/1732 [==============================] - 1s 296us/step


[1.3824796417844214, 0.4491916859122402]

In [68]:
def pos_tag2(sentences):
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    sentence = " ".join(tagger.morphs(sentences))
    sentences_pos.append(sentence)
    
  
    return sentences_pos

In [150]:
sentence ='다이어트 다이어트 다이어트 때문에 너무 고민입니다. 몸매가 살이 잘 찌는 편이라 맨날 뚱뚱해요 어떻게 해요'

test_sen = pos_tag2(sentence)

In [151]:
test= test_sen.pop()
test
    

'다이어트 다이어트 다이어트 때문 에 너무 고민 입니다 . 몸매 가 살이 잘 찌는 편이 라 맨날 뚱뚱해요 어떻게 해 요'

In [152]:
test= test.split()

In [153]:
def n_to_int2(x):
    pad_index = []
    for i in x:
        if i in noums_to_index.keys():
            pad_index.append(noums_to_index[i])
    return pad_index


In [154]:
test= n_to_int2(test)

In [155]:
test

[1243, 1243, 1243, 3360, 6182, 8723, 7637, 656, 8453, 880, 6156, 2297, 2401]

In [156]:
test= pad_sequences([test], maxlen=70)

In [157]:
test

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1243, 1243, 1243, 3360, 6182, 8723, 7637,  656, 8453,
         880, 6156, 2297, 2401]])

In [158]:
model1.predict_classes(test)

array([3], dtype=int64)